In [8]:
import collections, math, random, functools

In [9]:
def entropy(class_probabilities):
    #Obliczanie entropii na podstawie listy prawdopodobienstw klas.
    return sum(-p * math.log(p,2) 
               for p in class_probabilities 
               if p) #Ignoruj zerowe prawdopodobieństwo

def class_probabilities(labels):
    total_count = len(labels)
    return [count / total_count for count in collections.Counter(labels).values()]

def data_entropy(labeled_data):
    labels = [label for _,label in labeled_data]
    probabilities = class_probabilities(labels)
    return entropy(probabilities)

def partition_entropy(subsets):
    #Określanie entropii podziału danych na podzbiory, które mają forme listy list danych z etykietami
    total_count = sum(len(subset) for subset in subsets)
    return sum (data_entropy(subset) * len(subset) / total_count for subset in subsets)

In [10]:
def partition_by(inputs,attribute):
    groups = collections.defaultdict(list)
    for i in inputs:
        key = i[0][attribute]
        groups[key].append(i)
    return groups

def partition_entropy_by(inputs,attribute):
    #Obliczanie entropii danego podziału
    partitions = partition_by(inputs,attribute)
    return partition_entropy(partitions.values())

def classify(tree,input):
    #Klasyfikacja danych wejsciowych za pomocą drzewa decyzyjnego
    if tree in [True,False]:
        return tree
    
    attribute,subtree_dict = tree
    subtree_key = input.get(attribute)
    if subtree_key not in subtree_dict:
        subtree_key = None
        
    subtree = subtree_dict[subtree_key]
    return classify(subtree,input)


In [11]:
def build_tree_id3(inputs, split_candidates=None):
    #Rezprezentacja drzewa decyzyjnego na podstawie zbioru treningowego
    if split_candidates is None:
        split_candidates = inputs[0][0].keys()
        
    num_inputs = len(inputs)
    num_trues = len([label for item,label in inputs if label])
    num_falses = num_inputs - num_trues
    
    if num_trues == 0:
        return False
    if num_falses == 0:
        return True
    if not split_candidates:
        return num_trues >= num_falses
    
    best_attribute = min(split_candidates,
        key=functools.partial(partition_entropy_by, inputs))

    partitions = partition_by(inputs, best_attribute)
    new_candidates = [a for a in split_candidates
                      if a != best_attribute]

    subtrees = { attribute : build_tree_id3(subset, new_candidates) for attribute, subset in partitions.items() }

    subtrees[None] = num_trues > num_falses

    return (best_attribute, subtrees)

In [12]:

inputs = [
    ({'level':'Senior','lang':'Java','tweets':'no','phd':'no'},   False),
    ({'level':'Senior','lang':'Java','tweets':'no','phd':'yes'},  False),
    ({'level':'Mid','lang':'Python','tweets':'no','phd':'no'},     True),
    ({'level':'Junior','lang':'Python','tweets':'no','phd':'no'},  True),
    ({'level':'Junior','lang':'R','tweets':'yes','phd':'no'},      True),
    ({'level':'Junior','lang':'R','tweets':'yes','phd':'yes'},    False),
    ({'level':'Mid','lang':'R','tweets':'yes','phd':'yes'},        True),
    ({'level':'Senior','lang':'Python','tweets':'no','phd':'no'}, False),
    ({'level':'Senior','lang':'R','tweets':'yes','phd':'no'},      True),
    ({'level':'Junior','lang':'Python','tweets':'yes','phd':'no'}, True),
    ({'level':'Senior','lang':'Python','tweets':'yes','phd':'yes'},True),
    ({'level':'Mid','lang':'Python','tweets':'no','phd':'yes'},    True),
    ({'level':'Mid','lang':'Java','tweets':'yes','phd':'no'},      True),
    ({'level':'Junior','lang':'Python','tweets':'no','phd':'yes'},False)
]
    
tree = build_tree_id3(inputs)  
for key in ['level','lang','tweets','phd']:
    print(key, partition_entropy_by(inputs, key))
print()
    
senior_inputs = [(input, label)
                 for input, label in inputs if input["level"] == "Senior"]
    
for key in ['lang', 'tweets', 'phd']:
    print(key, partition_entropy_by(senior_inputs, key))
print()   
    
print("tree:\n", tree)
print()

print("|lvl: Juniro | lang: Java | tweets: yes | phd: yes|: ",classify(tree,
               { "level" : "Junior",
              "lang" : "Java",
              "tweets" : "yes",
              "phd" : "no"} ))

print("|lvl: Juniro | lang: Java | tweets: yes | phd: yes|: ", classify(tree,
               { "level" : "Junior",
                 "lang" : "Java",
                 "tweets" : "yes",
                 "phd" : "yes"} ))

print("|lvl: Intern|: ", classify(tree, { "level" : "Intern" } ))
print("|lvl: Senior|: ", classify(tree, { "level" : "Senior" } ))

level 0.6935361388961919
lang 0.8601317128547441
tweets 0.7884504573082896
phd 0.8921589282623617

lang 0.4
tweets 0.0
phd 0.9509775004326938

tree:
 ('level', {'Senior': ('tweets', {'no': False, 'yes': True, None: False}), 'Mid': True, 'Junior': ('phd', {'no': True, 'yes': False, None: True}), None: True})

|lvl: Juniro | lang: Java | tweets: yes | phd: yes|:  True
|lvl: Juniro | lang: Java | tweets: yes | phd: yes|:  False
|lvl: Intern|:  True
|lvl: Senior|:  False


In [16]:
inputs_20 = [
    ({'level':'Senior','lang':'Java','tweets':'no','phd':'no'},   False),
    ({'level':'Junior','lang':'Python','tweets':'no','phd':'yes'},False),
    ({'level':'Senior','lang':'Java','tweets':'no','phd':'yes'},  False),
    ({'level':'Mid','lang':'Java','tweets':'yes','phd':'no'},      True),
    ({'level':'Junior','lang':'Python','tweets':'no','phd':'no'},  True),
    ({'level':'Mid','lang':'Python','tweets':'no','phd':'no'},     True),
    ({'level':'Senior','lang':'Python','tweets':'yes','phd':'yes'},True),
    ({'level':'Mid','lang':'Python','tweets':'no','phd':'yes'},    True),
    ({'level':'Junior','lang':'R','tweets':'yes','phd':'no'},      True),
    ({'level':'Senior','lang':'R','tweets':'yes','phd':'no'},      True),
    ({'level':'Junior','lang':'R','tweets':'yes','phd':'yes'},    False),
    ({'level':'Senior','lang':'Python','tweets':'no','phd':'no'}, False),
    ({'level':'Senior','lang':'Java','tweets':'yes','phd':'no'},   False),
    ({'level':'Junior','lang':'Python','tweets':'yes','phd':'no'},False),
    ({'level':'Senior','lang':'Java','tweets':'no','phd':'no'},  False),
    ({'level':'Mid','lang':'Java','tweets':'no','phd':'no'},      True),
    ({'level':'Junior','lang':'Python','tweets':'yes','phd':'no'},  True),
    ({'level':'Mid','lang':'Python','tweets':'yes','phd':'yes'},     True),
    ({'level':'Senior','lang':'Python','tweets':'yes','phd':'no'},True),
    ({'level':'Mid','lang':'Python','tweets':'no','phd':'no'},    True)
]

tree_v2 = build_tree_id3(inputs_20)  
for key in ['level','lang','tweets','phd']:
    print(key, partition_entropy_by(inputs_v2, key))
print()
print("tree:\n",tree_v2)

level 0.681773601169986
lang 0.8781761404668604
tweets 0.9406454496153464
phd 0.9582001710694417

tree:
 ('level', {'Senior': ('lang', {'Java': False, 'Python': ('tweets', {'yes': True, 'no': False, None: True}), 'R': True, None: False}), 'Junior': ('phd', {'yes': False, 'no': ('lang', {'Python': ('tweets', {'no': True, 'yes': True, None: True}), 'R': True, None: True}), None: False}), 'Mid': True, None: True})


In [17]:
inputs_50 = [
    ({'level':'Senior','lang':'Java','tweets':'no','phd':'no'},   False),
    ({'level':'Junior','lang':'Python','tweets':'no','phd':'yes'},False),
    ({'level':'Senior','lang':'Java','tweets':'no','phd':'yes'},  False),
    ({'level':'Mid','lang':'Java','tweets':'yes','phd':'no'},      True),
    ({'level':'Junior','lang':'Python','tweets':'no','phd':'no'},  True),
    ({'level':'Mid','lang':'Python','tweets':'no','phd':'no'},     True),
    ({'level':'Senior','lang':'Python','tweets':'no','phd':'yes'},      True),
    ({'level':'Junior','lang':'R','tweets':'yes','phd':'no'},    False),
    ({'level':'Senior','lang':'Python','tweets':'yes','phd':'no'}, False),
    ({'level':'Senior','lang':'Java','tweets':'no','phd':'yes'},   False),
    ({'level':'Junior','lang':'Python','tweets':'no','phd':'yes'},False),
    ({'level':'Senior','lang':'Java','tweets':'yes','phd':'yes'},  False),
    ({'level':'Mid','lang':'Java','tweets':'no','phd':'yes'},      True),
    ({'level':'Junior','lang':'Python','tweets':'no','phd':'yes'},  True),
    ({'level':'Mid','lang':'Python','tweets':'yes','phd':'no'},     True),
    ({'level':'Senior','lang':'Python','tweets':'yes','phd':'no'},True),
    ({'level':'Senior','lang':'Python','tweets':'yes','phd':'yes'},True),
    ({'level':'Mid','lang':'Python','tweets':'no','phd':'yes'},    True),
    ({'level':'Junior','lang':'R','tweets':'yes','phd':'no'},      True),
    ({'level':'Senior','lang':'R','tweets':'yes','phd':'no'},      True),
    ({'level':'Junior','lang':'R','tweets':'yes','phd':'yes'},    False),
    ({'level':'Senior','lang':'Python','tweets':'no','phd':'no'}, False),
    ({'level':'Senior','lang':'Java','tweets':'yes','phd':'no'},   False),
    ({'level':'Junior','lang':'Python','tweets':'yes','phd':'no'},False),
    ({'level':'Senior','lang':'Java','tweets':'no','phd':'no'},  False),
    ({'level':'Mid','lang':'Java','tweets':'no','phd':'no'},      True),
    ({'level':'Junior','lang':'Python','tweets':'yes','phd':'no'},  True),
    ({'level':'Mid','lang':'Python','tweets':'yes','phd':'yes'},     True),
    ({'level':'Senior','lang':'Python','tweets':'yes','phd':'no'},True),
    ({'level':'Mid','lang':'Python','tweets':'no','phd':'no'},    True),
    ({'level':'Senior','lang':'Java','tweets':'no','phd':'no'},   False),
    ({'level':'Junior','lang':'Python','tweets':'no','phd':'yes'},False),
    ({'level':'Senior','lang':'Java','tweets':'no','phd':'yes'},  False),
    ({'level':'Mid','lang':'Java','tweets':'yes','phd':'no'},      True),
    ({'level':'Junior','lang':'Python','tweets':'no','phd':'no'},  True),
    ({'level':'Mid','lang':'Python','tweets':'no','phd':'no'},     True),
    ({'level':'Senior','lang':'Python','tweets':'yes','phd':'yes'},True),
    ({'level':'Mid','lang':'Python','tweets':'no','phd':'yes'},    True),
    ({'level':'Junior','lang':'R','tweets':'yes','phd':'no'},      True),
    ({'level':'Senior','lang':'R','tweets':'yes','phd':'no'},      True),
    ({'level':'Junior','lang':'R','tweets':'yes','phd':'yes'},    False),
    ({'level':'Senior','lang':'Python','tweets':'no','phd':'no'}, False),
    ({'level':'Senior','lang':'Java','tweets':'yes','phd':'no'},   False),
    ({'level':'Junior','lang':'Python','tweets':'yes','phd':'no'},False),
    ({'level':'Senior','lang':'Java','tweets':'no','phd':'no'},  False),
    ({'level':'Mid','lang':'Java','tweets':'no','phd':'no'},      True),
    ({'level':'Junior','lang':'Python','tweets':'yes','phd':'no'},  True),
    ({'level':'Mid','lang':'Python','tweets':'yes','phd':'yes'},     True),
    ({'level':'Senior','lang':'Python','tweets':'yes','phd':'no'},True),
    ({'level':'Mid','lang':'Python','tweets':'no','phd':'no'},    True)
]

tree_v3 = build_tree_id3(inputs_50)  
for key in ['level','lang','tweets','phd']:
    print(key, partition_entropy_by(inputs_v3, key))
print()
print("tree:\n",tree_v3)

level 0.7016964804448863
lang 0.8967682076584556
tweets 0.970764362625347
phd 0.9708377618719776

tree:
 ('level', {'Senior': ('lang', {'Java': False, 'Python': ('phd', {'yes': True, 'no': ('tweets', {'yes': True, 'no': False, None: False}), None: True}), 'R': True, None: False}), 'Junior': ('phd', {'yes': ('lang', {'Python': ('tweets', {'no': False, None: False}), 'R': False, None: False}), 'no': ('tweets', {'no': True, 'yes': ('lang', {'R': True, 'Python': True, None: True}), None: True}), None: False}), 'Mid': True, None: True})
